Hypertuning is finding optimal parameters for the model

In [95]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

iris = load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### traditional approach
problem with this is the score changes as the train and test dataset changes

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2)

In [97]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.9333333333333333

so, to solve varying model score issue, we use cross validation where we take average of multiple models with different train and test datasets
cross_val_score() method implements the CrossValidation() on the provided dataset and gives the array of all the models with different datasets

In [98]:
from sklearn.model_selection import cross_val_score

In [99]:
cross_val_score(SVC(kernel='linear', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [100]:
cross_val_score(SVC(kernel='rbf', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [101]:
cross_val_score(SVC(kernel='rbf', C=20, gamma='auto'), iris.data, iris.target, cv=5)


array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

manual way of hyperturnig

In [102]:
kernels = ['rbf', 'linear']
C = [1, 10, 20]
avg_score = {}
for kval in kernels:
	for cval in C:
		cv_score = cross_val_score(SVC(kernel=kval, C=cval, gamma='auto'), iris.data, iris.target, cv=5)
		avg_score[kval + "_" + str(cval)] = np.average(cv_score)

avg_score

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

### using GridSearchCV to automate Hypertuning

In [103]:
from sklearn.model_selection import GridSearchCV

classifier = GridSearchCV(SVC(gamma='auto'), {
	'C': [1, 10, 20],
	'kernel' : ['rbf', 'linear']
}, cv=5, return_train_score=False)

classifier.fit(iris.data, iris.target)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']})

In [104]:
classifier.cv_results_	# very hard to read

{'mean_fit_time': array([0.00104532, 0.00084114, 0.0009254 , 0.00082736, 0.00090652,
        0.00083313]),
 'std_fit_time': array([8.68250416e-05, 1.32985430e-05, 1.71313994e-05, 2.82299679e-05,
        2.19300514e-05, 1.30905479e-05]),
 'mean_score_time': array([0.0007412 , 0.00067077, 0.00069814, 0.00065365, 0.00069013,
        0.00065284]),
 'std_score_time': array([5.88157711e-05, 3.26763948e-06, 8.18216994e-06, 5.15248972e-06,
        8.44365381e-06, 2.77457771e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [105]:
results = pd.DataFrame(classifier.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001045,0.000087,0.000741,0.000059,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000841,0.000013,0.000671,0.000003,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000925,0.000017,0.000698,0.000008,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000827,0.000028,0.000654,0.000005,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000907,0.000022,0.000690,0.000008,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000833,0.000013,0.000653,0.000028,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [106]:
results[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [107]:
classifier.best_score_

0.9800000000000001

In [108]:
classifier.best_estimator_

SVC(C=1, gamma='auto')

In [109]:
classifier.best_params_

{'C': 1, 'kernel': 'rbf'}

### RandomizedSearchCV
since GridSearchCV tries all possible combinations of parameters it has a lot of computational overhead

RandomizedSearchCV tries randome parameters and not all parameters

the values of parameters will be selected randomly

In [110]:
classifier.cv_results_	# very hard to read

{'mean_fit_time': array([0.00104532, 0.00084114, 0.0009254 , 0.00082736, 0.00090652,
        0.00083313]),
 'std_fit_time': array([8.68250416e-05, 1.32985430e-05, 1.71313994e-05, 2.82299679e-05,
        2.19300514e-05, 1.30905479e-05]),
 'mean_score_time': array([0.0007412 , 0.00067077, 0.00069814, 0.00065365, 0.00069013,
        0.00065284]),
 'std_score_time': array([5.88157711e-05, 3.26763948e-06, 8.18216994e-06, 5.15248972e-06,
        8.44365381e-06, 2.77457771e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [111]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma='auto'), {
	'C':[1, 10, 20],
	'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False, n_iter=2)	# n_iter=2 tells that we only want to try 2 combinations

rs.fit(iris.data, iris.target)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']})

In [112]:
pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,10,rbf,0.980000


In [113]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [114]:

model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear'),
        'params': {
            'C': [1,5,10]
        }
    }
}


In [116]:
scores = []

for model_name, mp in model_params.items():
	clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
	clf.fit(iris.data, iris.target)
	scores.append({
		'model':model_name,
		'best_score': clf.best_score_,
		'best_params':clf.best_params_
	})

In [119]:
scores = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
scores

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
